In [1]:
import tensorflow as tf
import numpy as np

In [32]:
def var_summary(var_list, printer=None):
    printer = tf.constant(0) if printer is None else printer
    printer = tf.Print(input_=printer, data=[tf.reduce_mean(v) for v in var_list])
    return printer

def categorical_sample_logits(X):
    # https://github.com/tensorflow/tensorflow/issues/456
    U = tf.random_uniform(tf.shape(X))
    return tf.argmax(X - tf.log(-tf.log(U)), axis=1)
        

In [39]:
class MLP(object):
    def __init__(self, name, hidden_sizes, in_dim=32, out_dim=5, trainable=False, lr=0.001, out_type='regress'):
        inp = tf.placeholder(dtype=tf.float32,shape=[None,in_dim])
        x = inp
        with tf.variable_scope(name):
            for i, si in enumerate(hidden_sizes):
                x = tf.layers.dense(inputs=x, activation=tf.nn.relu, name='h{}'.format(i), units=si,trainable=trainable)
            if out_type== 'regress':
                pred = tf.layers.dense(inputs=x, activation=None, units=out_dim, name='output', trainable=trainable)
            else:
                logits = tf.layers.dense(inputs=x, activation=None, units=out_dim, name='logits', trainable=trainable)
                pred = tf.argmax(tf.nn.softmax(logits=logits),axis=1)
                pred = tf.one_hot(pred, depth=out_dim)
                
            def predict(sess, X):
                return sess.run(pred, feed_dict={inp:X})
            self.predict = predict
            my_vars = tf.get_collection(key=tf.GraphKeys.GLOBAL_VARIABLES, scope=name)
            if trainable:
                y = tf.placeholder(dtype=tf.float32, shape=[None, out_dim])
                if out_type== 'regress':
                    loss = tf.reduce_mean(tf.square(y-pred))
                else:
                    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=logits))
                optimizer = tf.train.AdamOptimizer(learning_rate=lr)
                grads_clipped = [tf.clip_by_value(g, -1., 1.) for g,_ in optimizer.compute_gradients(loss, my_vars)]
                opt_op = optimizer.apply_gradients(zip(grads_clipped, my_vars))
                
                def run_opt(sess, X, labels):
                    loss_val, _ = sess.run([loss, opt_op], feed_dict={inp:X, y:labels})
                    return loss_val
                self.run_opt = run_opt
        
        printables = var_summary(my_vars)
        def printer(sess, X):
            sess.run(printables, feed_dict={inp:X})
            return
        self.printer = printer

In [40]:
ITERS = 1000000
BATCH = 128
IN_DIM = 256
OUT_TYPE = 'classification'

In [41]:
tf.reset_default_graph()
net = MLP('target', hidden_sizes=[128, 128, 64], in_dim=IN_DIM,out_type=OUT_TYPE)

In [42]:
net2 = MLP('learner', hidden_sizes=[256,128,64,32], trainable=True, in_dim=IN_DIM,out_type=OUT_TYPE)

In [43]:
def validate(net, net2, z, sess, out_type):
    target = net.predict(X=z, sess=sess)
    pred = net2.predict(X=z, sess=sess)
    if out_type == 'regress':
        target = target.reshape(-1)
        pred = pred.reshape(-1)
        return 1- (np.var(target-pred)/ (np.var(target)+1e-8))
    elif out_type == 'classification':
        target = np.argmax(target, axis=1)
        pred = np.argmax(pred, axis=1)
        return np.mean(np.array(target == pred,dtype=np.float32))
    else:
        raise NotImplementedError
        

In [44]:
val_acc = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(ITERS):
        z = np.random.normal(size=[BATCH, IN_DIM])
        if i % 50 == 0:
            var_act = validate(net, net2, z, sess, out_type=OUT_TYPE)
            val_acc.append(var_act)
            #net.printer(sess=sess, X=z)
            print("Iteration {}. Var accouned for {}.".format(i, var_act))
        else:
            loss_val = net2.run_opt(X=z, labels=net.predict(X=z, sess=sess), sess=sess)
            if i % 20 ==0:
                print("Iteration {}. Loss value {}".format(i, loss_val))


        
        
    
    

Iteration 0. Var accouned for 0.1171875.
Iteration 20. Loss value 0.7877920269966125
Iteration 40. Loss value 0.7600010633468628
Iteration 50. Var accouned for 0.7265625.
Iteration 60. Loss value 0.7278940677642822
Iteration 80. Loss value 0.876861572265625
Iteration 100. Var accouned for 0.7109375.
Iteration 120. Loss value 0.7967367172241211
Iteration 140. Loss value 0.792167067527771
Iteration 150. Var accouned for 0.7734375.
Iteration 160. Loss value 0.8331837058067322
Iteration 180. Loss value 0.7279686331748962
Iteration 200. Var accouned for 0.8203125.
Iteration 220. Loss value 0.6417995691299438
Iteration 240. Loss value 0.5981432199478149
Iteration 250. Var accouned for 0.8203125.
Iteration 260. Loss value 0.7488672733306885
Iteration 280. Loss value 0.713714063167572
Iteration 300. Var accouned for 0.7421875.
Iteration 320. Loss value 0.8388625979423523
Iteration 340. Loss value 0.791343629360199
Iteration 350. Var accouned for 0.7578125.
Iteration 360. Loss value 0.776053249

Iteration 3100. Var accouned for 0.765625.
Iteration 3120. Loss value 0.6608594655990601
Iteration 3140. Loss value 0.6031953692436218
Iteration 3150. Var accouned for 0.765625.
Iteration 3160. Loss value 0.5374088883399963
Iteration 3180. Loss value 0.6448269486427307
Iteration 3200. Var accouned for 0.7265625.
Iteration 3220. Loss value 0.4784104824066162
Iteration 3240. Loss value 0.5396763682365417
Iteration 3250. Var accouned for 0.8046875.
Iteration 3260. Loss value 0.4865005314350128
Iteration 3280. Loss value 0.5906609296798706
Iteration 3300. Var accouned for 0.765625.
Iteration 3320. Loss value 0.6200560331344604
Iteration 3340. Loss value 0.7198143005371094
Iteration 3350. Var accouned for 0.8203125.
Iteration 3360. Loss value 0.6789935827255249
Iteration 3380. Loss value 0.5268982648849487
Iteration 3400. Var accouned for 0.7890625.
Iteration 3420. Loss value 0.5113075971603394
Iteration 3440. Loss value 0.5237326622009277
Iteration 3450. Var accouned for 0.796875.
Iteratio

Iteration 6160. Loss value 0.5107405781745911
Iteration 6180. Loss value 0.4951896071434021
Iteration 6200. Var accouned for 0.8203125.
Iteration 6220. Loss value 0.4745790362358093
Iteration 6240. Loss value 0.6516065001487732
Iteration 6250. Var accouned for 0.75.
Iteration 6260. Loss value 0.4608960747718811
Iteration 6280. Loss value 0.49192097783088684
Iteration 6300. Var accouned for 0.75.
Iteration 6320. Loss value 0.5331624150276184
Iteration 6340. Loss value 0.4948517978191376
Iteration 6350. Var accouned for 0.6875.
Iteration 6360. Loss value 0.5440801978111267
Iteration 6380. Loss value 0.5689107179641724
Iteration 6400. Var accouned for 0.765625.
Iteration 6420. Loss value 0.47315457463264465
Iteration 6440. Loss value 0.4061460494995117
Iteration 6450. Var accouned for 0.8203125.
Iteration 6460. Loss value 0.42569392919540405
Iteration 6480. Loss value 0.5746078491210938
Iteration 6500. Var accouned for 0.7890625.
Iteration 6520. Loss value 0.4489842653274536
Iteration 654

Iteration 9220. Loss value 0.4753332734107971
Iteration 9240. Loss value 0.4863220453262329
Iteration 9250. Var accouned for 0.8359375.
Iteration 9260. Loss value 0.5557668209075928
Iteration 9280. Loss value 0.6203117966651917
Iteration 9300. Var accouned for 0.8046875.
Iteration 9320. Loss value 0.4249008893966675
Iteration 9340. Loss value 0.5135577321052551
Iteration 9350. Var accouned for 0.84375.
Iteration 9360. Loss value 0.5898844003677368
Iteration 9380. Loss value 0.3994739055633545
Iteration 9400. Var accouned for 0.78125.
Iteration 9420. Loss value 0.5360596179962158
Iteration 9440. Loss value 0.5305309295654297
Iteration 9450. Var accouned for 0.828125.
Iteration 9460. Loss value 0.5585441589355469
Iteration 9480. Loss value 0.5859429240226746
Iteration 9500. Var accouned for 0.8671875.
Iteration 9520. Loss value 0.4217236042022705
Iteration 9540. Loss value 0.4901789426803589
Iteration 9550. Var accouned for 0.8046875.
Iteration 9560. Loss value 0.5023500919342041
Iterati

Iteration 12200. Var accouned for 0.7890625.
Iteration 12220. Loss value 0.5481961965560913
Iteration 12240. Loss value 0.46390122175216675
Iteration 12250. Var accouned for 0.8046875.
Iteration 12260. Loss value 0.5210322737693787
Iteration 12280. Loss value 0.4205753207206726
Iteration 12300. Var accouned for 0.828125.
Iteration 12320. Loss value 0.48861950635910034
Iteration 12340. Loss value 0.551929235458374
Iteration 12350. Var accouned for 0.8046875.
Iteration 12360. Loss value 0.5901784896850586
Iteration 12380. Loss value 0.48724430799484253
Iteration 12400. Var accouned for 0.7890625.
Iteration 12420. Loss value 0.4677989184856415
Iteration 12440. Loss value 0.4473766088485718
Iteration 12450. Var accouned for 0.828125.
Iteration 12460. Loss value 0.38193395733833313
Iteration 12480. Loss value 0.3484100103378296
Iteration 12500. Var accouned for 0.796875.
Iteration 12520. Loss value 0.39277926087379456
Iteration 12540. Loss value 0.3465772867202759
Iteration 12550. Var accou

...

Iteration 50840. Loss value 0.3885228633880615
Iteration 50850. Var accouned for 0.78125.
Iteration 50860. Loss value 0.4541684091091156
Iteration 50880. Loss value 0.5353245139122009
Iteration 50900. Var accouned for 0.859375.
Iteration 50920. Loss value 0.5407146215438843
Iteration 50940. Loss value 0.5067856311798096
Iteration 50950. Var accouned for 0.8203125.
Iteration 50960. Loss value 0.5142751336097717
Iteration 50980. Loss value 0.38020700216293335
Iteration 51000. Var accouned for 0.8125.
Iteration 51020. Loss value 0.542180061340332
Iteration 51040. Loss value 0.42434942722320557
Iteration 51050. Var accouned for 0.796875.
Iteration 51060. Loss value 0.40200263261795044
Iteration 51080. Loss value 0.5807971954345703
Iteration 51100. Var accouned for 0.8125.
Iteration 51120. Loss value 0.41599196195602417
Iteration 51140. Loss value 0.4306230843067169
Iteration 51150. Var accouned for 0.78125.
Iteration 51160. Loss value 0.41457462310791016
Iteration 51180. Loss value 0.40167

Iteration 53800. Var accouned for 0.8203125.
Iteration 53820. Loss value 0.41260379552841187
Iteration 53840. Loss value 0.4086882174015045
Iteration 53850. Var accouned for 0.859375.
Iteration 53860. Loss value 0.30141812562942505
Iteration 53880. Loss value 0.45166653394699097
Iteration 53900. Var accouned for 0.8125.
Iteration 53920. Loss value 0.5548677444458008
Iteration 53940. Loss value 0.505715012550354
Iteration 53950. Var accouned for 0.8671875.
Iteration 53960. Loss value 0.4779902994632721
Iteration 53980. Loss value 0.3844417631626129
Iteration 54000. Var accouned for 0.765625.
Iteration 54020. Loss value 0.3904811441898346
Iteration 54040. Loss value 0.3503050208091736
Iteration 54050. Var accouned for 0.875.
Iteration 54060. Loss value 0.4833385944366455
Iteration 54080. Loss value 0.5027005672454834
Iteration 54100. Var accouned for 0.8671875.
Iteration 54120. Loss value 0.4124915301799774
Iteration 54140. Loss value 0.5486530661582947
Iteration 54150. Var accouned for 

Iteration 56760. Loss value 0.32143634557724
Iteration 56780. Loss value 0.4738084673881531
Iteration 56800. Var accouned for 0.8203125.
Iteration 56820. Loss value 0.4697827100753784
Iteration 56840. Loss value 0.3825860023498535
Iteration 56850. Var accouned for 0.8359375.
Iteration 56860. Loss value 0.3510645031929016
Iteration 56880. Loss value 0.37077194452285767
Iteration 56900. Var accouned for 0.9140625.
Iteration 56920. Loss value 0.3580074906349182
Iteration 56940. Loss value 0.34163519740104675
Iteration 56950. Var accouned for 0.8046875.
Iteration 56960. Loss value 0.5375048518180847
Iteration 56980. Loss value 0.37932059168815613
Iteration 57000. Var accouned for 0.734375.
Iteration 57020. Loss value 0.42210036516189575
Iteration 57040. Loss value 0.3274369239807129
Iteration 57050. Var accouned for 0.828125.
Iteration 57060. Loss value 0.46706274151802063
Iteration 57080. Loss value 0.27374035120010376
Iteration 57100. Var accouned for 0.84375.
Iteration 57120. Loss value

Iteration 59750. Var accouned for 0.828125.
Iteration 59760. Loss value 0.42526480555534363
Iteration 59780. Loss value 0.5920917987823486
Iteration 59800. Var accouned for 0.8125.
Iteration 59820. Loss value 0.2718263268470764
Iteration 59840. Loss value 0.5214115381240845
Iteration 59850. Var accouned for 0.8359375.
Iteration 59860. Loss value 0.3595117926597595
Iteration 59880. Loss value 0.4542270302772522
Iteration 59900. Var accouned for 0.875.
Iteration 59920. Loss value 0.3943450450897217
Iteration 59940. Loss value 0.5558028221130371
Iteration 59950. Var accouned for 0.8125.
Iteration 59960. Loss value 0.40412425994873047
Iteration 59980. Loss value 0.4922659397125244
Iteration 60000. Var accouned for 0.828125.
Iteration 60020. Loss value 0.45371025800704956
Iteration 60040. Loss value 0.3948284089565277
Iteration 60050. Var accouned for 0.8203125.
Iteration 60060. Loss value 0.4537428617477417
Iteration 60080. Loss value 0.48321443796157837
Iteration 60100. Var accouned for 0

KeyboardInterrupt: 

In [46]:
np.save(file='log',allow_pickle=True,arr=val_acc)
    